In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os



In [2]:
DATA_DIR = '/home/chicm/ml/kgdata/species'
TRAIN_DIR = DATA_DIR + '/train'
RESULT_DIR = DATA_DIR + '/results'

TRAIN_FEAT = RESULT_DIR + '/train_feats.dat'
VAL_FEAT = RESULT_DIR + '/val_feats.dat'


#VAL_DIR = DATA_DIR + '/val-224'

batch_size = 64

In [3]:
df_train = pd.read_csv(DATA_DIR+'/train_labels.csv')
master = df_train
master.head()

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1


In [4]:
img_path = TRAIN_DIR+'/'

In [5]:
y = []
file_paths = []
for i in range(len(master)):
    file_paths.append( TRAIN_DIR + '/' + str(master.ix[i][0]) +'.jpg' )
    y.append(master.ix[i][1])
y = np.array(y)
print(y.shape)

(2295,)


In [6]:
print(file_paths[:5])

['/home/chicm/ml/kgdata/species/train/1.jpg', '/home/chicm/ml/kgdata/species/train/2.jpg', '/home/chicm/ml/kgdata/species/train/3.jpg', '/home/chicm/ml/kgdata/species/train/4.jpg', '/home/chicm/ml/kgdata/species/train/5.jpg']


In [7]:
#image reseize & centering & crop 

def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized


x = []
for i, file_path in enumerate(file_paths):
    #read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    x.append(img)

x = np.array(x)
print(x.shape)


(2295, 224, 224, 3)


In [8]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")
img_path = DATA_DIR+"/test/"

test_names = []
file_paths = []

for i in range(len(sample_submission)):
    test_names.append(sample_submission.ix[i][0])
    file_paths.append( img_path + str(int(sample_submission.ix[i][0])) +'.jpg' )
    
test_names = np.array(test_names)
print(test_names.shape)

(1531,)


In [9]:
test_images = []
for file_path in file_paths:
    #read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    test_images.append(img)
    
    path, ext = os.path.splitext( os.path.basename(file_paths[0]) )

test_images = np.array(test_images)

## split train and val

In [10]:
data_num = len(y)
random_index = np.random.permutation(data_num)

x_shuffle = []
y_shuffle = []
for i in range(data_num):
    x_shuffle.append(x[random_index[i]])
    y_shuffle.append(y[random_index[i]])
    
x = np.array(x_shuffle) 
y = np.array(y_shuffle)

In [11]:
val_split_num = int(round(0.2*len(y)))
x_train = x[val_split_num:]
y_train = y[val_split_num:]
x_test = x[:val_split_num]
y_test = y[:val_split_num]

print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_train (1836, 224, 224, 3)
y_train (1836,)
x_test (459, 224, 224, 3)
y_test (459,)


In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Use VGG

In [13]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense

img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

Using TensorFlow backend.


In [14]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

batch_size = 32
epochs = 50

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)


history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test), verbose=2,
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/50
23s - loss: 0.3204 - acc: 0.8646 - val_loss: 0.2383 - val_acc: 0.9041
Epoch 2/50
24s - loss: 0.2244 - acc: 0.9062 - val_loss: 0.1825 - val_acc: 0.9325
Epoch 3/50
23s - loss: 0.1891 - acc: 0.9319 - val_loss: 0.1829 - val_acc: 0.9325
Epoch 4/50
23s - loss: 0.1731 - acc: 0.9260 - val_loss: 0.2005 - val_acc: 0.9172
Epoch 5/50
23s - loss: 0.1441 - acc: 0.9410 - val_loss: 0.1600 - val_acc: 0.9368
Epoch 6/50
23s - loss: 0.1306 - acc: 0.9496 - val_loss: 0.1244 - val_acc: 0.9521
Epoch 7/50
23s - loss: 0.1447 - acc: 0.9435 - val_loss: 0.1053 - val_acc: 0.9586
Epoch 8/50
23s - loss: 0.1161 - acc: 0.9613 - val_loss: 0.1267 - val_acc: 0.9564
Epoch 9/50
23s - loss: 0.1039 - acc: 0.9605 - val_loss: 0.0998 - val_acc: 0.9651
Epoch 10/50
23s - loss: 0.0972 - acc: 0.9660 - val_loss: 0.1166 - val_acc: 0.9630
Epoch 11/50
23s - loss: 0.1062 - acc: 0.9635 - val_loss: 0.1130 - val_acc: 0.9630
Epoch 12/50
23s - loss: 0.0905 - acc: 0.9682 - val_loss: 0.0814 - val_acc: 0.9651
Epoch 13/50
23s - loss: 0

In [17]:
test_images = test_images.astype('float32')
test_images /= 255

In [18]:
model.load_weights('VGG16-transferlearning.model')
predictions = model.predict(test_images)

In [19]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")

for i, name in enumerate(test_names):
    sample_submission.loc[sample_submission['name'] == name, 'invasive'] = predictions[i]

sample_submission.to_csv("submit1.csv", index=False)